In [142]:
import numpy as np

In [184]:
train = open('../data/processed/train.word','r').readlines()
freqsf = open('../data/processed/freqs.txt', 'r').readlines()

In [185]:
s1s = [l.split('\t')[1].split(' ') for l in train]
s2s = [l.split('\t')[2].split(' ') for l in train]
labels = [int(l.split('\t')[3]) for l in train]
words = [s.split(' ')[0] for s in freqsf if s[0] != ' ']
freqs = [int(s.strip().split(' ')[1]) for s in freqsf  if s[0] != ' ']

In [186]:
count = [0,0]
for l in labels:
    count[l] += 1
print(count, float(count[0])/count[1])
ratio = float(count[0])/count[1]

[27676, 7735] 3.578021978021978


In [222]:
def analyze_boolean_feature(func, verbose=False):
    ratio = float(count[0])/count[1]
    eps = 1e-6
#     ratio = 1.
    n = 0
    results = [[0, 0], [0, 0]]
    hit = [0,0]
    cls = [0,0]
    for s1, s2, l in zip(s1s, s2s, labels):
        n += 1
        r = func(s1, s2, l)
        results[r][l] += 1
        hit[r] += 1
        cls[l] += 1
    PCF = np.zeros((2,2)) # P(C|F)
    PFC = np.zeros((2,2)) # P(F|C)
    for c in [0,1]:
        for f in [0,1]: 
            PCF[c,f] = results[f][c]/float(hit[f] + eps) 
    for c in [0,1]:
        for f in [0,1]: 
            PFC[f,c] = results[f][c]/float(cls[c] + eps) 
    if verbose:
        print("# C            0         1")
        print("# P(C|F=0)   {0:.3f}      {1:.3f}".format(PCF[0,0], PCF[1,0]))
        print("# P(C|F=1)   {0:.3f}      {1:.3f}".format(PCF[0,1], PCF[1,1]))
#         print("# Feature=0  {0:.3f}      {1:.3f}".format(results[0][0]/float(n), results[0][1]/float(n)*ratio))
#         print("# Feature=1  {0:.3f}      {1:.3f}".format(results[1][0]/float(n), results[1][1]/float(n)*ratio))  
        print("# P(F=0|C)   {0:.3f}      {1:.3f}".format(PFC[0,0], PFC[0,1]))
        print("# P(F=1|C)   {0:.3f}      {1:.3f}".format(PFC[1,0], PFC[1,1]))
    return PCF, PFC

In [225]:
def func(s1, s2, l):
    # 只在一个句子中有该词
    word = u'花呗' # 要注意同义词, 手动设置近义疑问词
    if (word in s1 and word not in s2) or (word in s2 and word not in s1):
#         if l == 1:
#             print(s1, s2)
        return 1
    else:
        return 0

analyze_boolean_feature(func, True)
# 花呗
#              0         1
# Feature=0  0.787      0.815
# Feature=1  0.931      0.265
# Feature=0  0.756      0.782
# Feature=1  0.038      0.011

# 借呗
#              0         1
# Feature=0  0.786      0.822
# Feature=1  0.946      0.208
# Feature=0  0.749      0.783
# Feature=1  0.044      0.010

# C            0         1
# P(C|F=0)   0.776      0.224
# P(C|F=1)   0.927      0.073
# P(F=0|C)   0.955      0.987
# P(F=1|C)   0.045      0.013


(array([[ 0.7758474 ,  0.92739521],
        [ 0.2241526 ,  0.07260479]]), array([[ 0.95523197,  0.9874596 ],
        [ 0.04476803,  0.0125404 ]]))

In [229]:
def build_func(word):
    def func(s1, s2, l):
        # 只在一个句子中有该词
        if (word in s1 and word not in s2) or (word in s2 and word not in s1):
            return 1
        else:
            return 0
    return func
cands = []
for w, f in zip(words, freqs):
    if f > 100:
        func = build_func(w)
        PCF, PFC = analyze_boolean_feature(func)
        if np.abs(PCF[0,0] - PCF[0,1]) > 0.1:
            print("{0}\t P(C=0|F=0)={1:.3f}, P(C=0|F=1)={2:.3f}, P(F=1|C=0)={3:.3f}, P(F=1|C=1)={4:.3f}"\
                                  .format(w, PCF[0,0], PCF[0,1], PFC[1,0], PFC[1,1]))
            cands.append(w)
        
    

花呗	 P(C=0|F=0)=0.776, P(C=0|F=1)=0.927, P(F=1|C=0)=0.045, P(F=1|C=1)=0.013
借呗	 P(C=0|F=0)=0.774, P(C=0|F=1)=0.956, P(F=1|C=0)=0.051, P(F=1|C=1)=0.008
分期	 P(C=0|F=0)=0.776, P(C=0|F=1)=0.891, P(F=1|C=0)=0.053, P(F=1|C=1)=0.023
多少	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.881, P(F=1|C=0)=0.025, P(F=1|C=1)=0.012
逾期	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.912, P(F=1|C=0)=0.026, P(F=1|C=1)=0.009
提前	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.934, P(F=1|C=0)=0.022, P(F=1|C=1)=0.005
银行卡	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.884, P(F=1|C=0)=0.018, P(F=1|C=1)=0.008
余额	 P(C=0|F=0)=0.779, P(C=0|F=1)=0.920, P(F=1|C=0)=0.022, P(F=1|C=1)=0.007
淘宝	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.894, P(F=1|C=0)=0.020, P(F=1|C=1)=0.009
临时	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.955, P(F=1|C=0)=0.010, P(F=1|C=1)=0.002
影响	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.918, P(F=1|C=0)=0.012, P(F=1|C=1)=0.004
利息	 P(C=0|F=0)=0.781, P(C=0|F=1)=0.886, P(F=1|C=0)=0.011, P(F=1|C=1)=0.005
信用卡	 P(C=0|F=0)=0.780, P(C=0|F=1)=0.946, P(F=1|C=0)=0.013, P(F=1|C=1)=0.003
手续费	 P(C=0|F=0)=0.781, 

In [230]:
cands

['花呗',
 '借呗',
 '分期',
 '多少',
 '逾期',
 '提前',
 '银行卡',
 '余额',
 '淘宝',
 '临时',
 '影响',
 '利息',
 '信用卡',
 '手续费',
 '扣',
 '是不是',
 '最低',
 '宝',
 '不是',
 '算',
 '身份证',
 '为啥',
 '积分',
 '电费',
 '充值',
 '密码',
 '免息',
 '红包',
 '限制',
 '二维码',
 '授权',
 '限额',
 '更改',
 '没到',
 '方式',
 '别人',
 '充',
 '评估',
 '单车',
 '微信',
 '兑换',
 '负',
 '卖家',
 '线下',
 '先',
 '返现',
 '超市',
 '六个月',
 '滴滴',
 '请',
 '双',
 '放款',
 '产生',
 '几个',
 '一期',
 '少',
 '不足',
 '绑',
 '没收',
 '邀请',
 '服务费',
 '店铺',
 '活动',
 '繁忙',
 '密付',
 '电脑',
 '必须',
 '全额',
 '多长时间',
 '津贴',
 '两次',
 '转到',
 '人脸识别',
 '按',
 '证明',
 '共享',
 '几点',
 '负面',
 '推迟',
 '低',
 '通讯录',
 '买家',
 '一起',
 '天猫',
 '一年',
 '学历',
 '几次',
 '口碑',
 '客户',
 '实体店',
 '领取']

In [163]:
def contains(words, l):
    for w in words:
        if w in l:
            return True
    return False

def func(s1, s2, l):
    # 只在一个句子中有该类词
    word = [u'借呗'] # 要注意同义词, 手动设置近义疑问词
    if (contains(word, s1) and not contains(word, s2)) or (contains(word, s2) and not contains(word, s1)):
#         if l == 1:
#             print(s1, s2)
        return 1
    else:
        return 0

analyze_boolean_feature(func)

In [214]:
def func(s1, s2, l):
    # 两个句子里都有该词
    word = u'退款'
    if word in s1 and word in s2:
        return 1
    else:
        return 0

analyze_boolean_feature(func, True)

# C            0         1
# P(C|F=0)   0.783      0.217
# P(C|F=1)   0.667      0.333
# P(F=0|C)   0.986      0.975
# P(F=1|C)   0.014      0.025


(array([[ 0.78345485,  0.66666667],
        [ 0.21654515,  0.33333333]]), array([[ 0.98619743,  0.97530705],
        [ 0.01380257,  0.02469295]]))

In [228]:
def build_func(word):
    def func(s1, s2, l):
        # 两个句子里都有该词
        if word in s1 and word in s2:
            return 1
        else:
            return 0
    return func
    
for w, f in zip(words, freqs):
    if f > 100:
        func = build_func(w)
        PCF, PFC = analyze_boolean_feature(func)
        if np.abs(PCF[0,0] - PCF[0,1]) > 0.1 and PFC[1,0] > 0.01:
            print("{0}\t P(C=0|F=0)={1:.3f}, P(C=0|F=1)={2:.3f}, P(F=1|C=0)={3:.3f}, P(F=1|C=1)={4:.3f}"\
                                  .format(w, PCF[0,0], PCF[0,1], PFC[1,0], PFC[1,1]))
        
    

在	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.662, P(F=1|C=0)=0.010, P(F=1|C=1)=0.018
退款	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.667, P(F=1|C=0)=0.014, P(F=1|C=1)=0.025
关闭	 P(C=0|F=0)=0.783, P(C=0|F=1)=0.644, P(F=1|C=0)=0.010, P(F=1|C=1)=0.020


In [206]:
def func(s1, s2, l):
    # 两个句子里都没有该词
    word = u'花呗'
    if word not in s1 and word not in s2:
        return 1
    else:
        return 0

analyze_boolean_feature(func, True)

# C            0         1
# P(C|F=0)   0.790      0.210
# P(C|F=1)   0.759      0.241
# P(F=0|C)   0.733      0.697
# P(F=1|C)   0.267      0.303


(array([[ 0.79011336,  0.75903491],
        [ 0.20988664,  0.24096509]]), array([[ 0.73287325,  0.69657401],
        [ 0.26712675,  0.30342599]]))